In [8]:
import pandas as pd

path = "https://github.com/queiyanglim/trading_algorithm/raw/master/oil_trading/data/oil_prices.csv"

df_pull = pd.read_csv(path, header=[0], index_col = 0)
df_pull = df_pull[["CO1 Comdty", "CL1 Comdty"]]
df_pull.index.name = "timestamp"
df_pull = df_pull.rename(columns = {"CO1 Comdty": "brent",
                                    "CL1 Comdty": "wti"})
df_pull

,brent,wti
timestamp,,
03/01/1990,21.69,23.68
04/01/1990,21.51,23.41
05/01/1990,22.00,23.08
08/01/1990,20.52,21.62
09/01/1990,20.14,22.07
...,...,...
03/03/2020,51.86,47.18
04/03/2020,51.13,46.78
05/03/2020,49.99,45.90
